<a href="https://colab.research.google.com/github/Dinmir331/Semester8_LB3/blob/main/S8_CV_LB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

3 лаба -- Дано видео, нужно его считать (на видео бегает мыш по полу), отделить мышь от пола и найти её центр на каждом кадре видеоряда используя код разумеется, а не покадровое разбиение с указанием где центр вручную.


# Импорт необходимых библиотек

In [ ]:
import numpy as np                  # Для работы с массивами и математическими операциями
import cv2                          # Основная библиотека для компьютерного зрения
import imageio                      # Для создания GIF-анимаций
from google.colab import drive      # Для подключения Google Drive

drive.mount('/content/drive', force_remount=True) # Монтируем Google Drive

Mounted at /content/drive


# Инициализия видеопотка и возврат параметров видео

In [ ]:
def initialize_video(video_path):

    cap = cv2.VideoCapture(video_path) # Открываем видеофайл
    if not cap.isOpened():
        raise ValueError(f"Ошибка открытия видео: {video_path}")
    # Определяем целевой размер для масштабирования (используется при обработке каждого кадра)
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return cap, original_width, original_height

# Обучение вычитателя фона на начальных кадрах

In [ ]:
def calculate_static_background(cap, initial_frames, learning_rate,median_blur,morph_iterations):
    ret, frame = cap.read()

    # Сбор кадров для медианного усреднения
    buffer = []
    for _ in range(initial_frames):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, median_blur)  # Медианное размытие
        buffer.append(gray)
        ret, frame = cap.read()
        if not ret:
            break

    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Сброс к началу

    # Вычисление медианного фона
    background = np.median(buffer, axis=0).astype(np.uint8)

    # Улучшение фона морфологией
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    background = cv2.morphologyEx(background, cv2.MORPH_CLOSE, kernel, iterations=morph_iterations)

    return background

# Создание трекера

In [ ]:
def csrt_tracking(frame, roi, tracker_state=None):
    """
    Реализация CSRT-подобного трекинга через корреляцию шаблонов
    :param frame: Текущий кадр
    :param roi: ROI-область в формате (x,y,w,h)
    :param tracker_state: Состояние трекера (для сохранения шаблона)
    :return: Новый bbox, состояние трекера, статус успеха
    """

    if tracker_state is None:
        # Инициализация трекера
        x, y, w, h = roi
        template = frame[y:y+h, x:x+w]
        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
        template_gray = cv2.GaussianBlur(template_gray, (7,7), 0)
        return {'template': template_gray}, (x,y,w,h), True

    # Обновление трекера
    template = tracker_state['template']
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (7,7), 0)

    # Поиск совпадений методом корреляции
    result = cv2.matchTemplate(frame_gray, template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

    if max_val < 0.5:  # Порог совпадения
        return None, None, False

    # Вычисление новых координат
    h_template, w_template = template.shape[:2]
    x, y = max_loc
    new_bbox = (x, y, w_template, h_template)

    # Обновление шаблона каждые 5 кадров
    if np.random.randint(5) == 0:
        new_template = frame[y:y+h_template, x:x+w_template]
        new_template_gray = cv2.cvtColor(new_template, cv2.COLOR_BGR2GRAY)
        new_template_gray = cv2.GaussianBlur(new_template_gray, (7,7), 0)
        tracker_state['template'] = new_template_gray



    return tracker_state, new_bbox, True

# Обработка кадров для поиска объекта

In [ ]:
def process_frame(frame, background, var_threshold, min_area, max_aspect_ratio):
    # Вычисление маски движения
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Преобразование в градации серого
    fgmask = cv2.absdiff(gray_frame, background)  # Разница с фоном
    _, fgmask = cv2.threshold(fgmask, var_threshold, 255, cv2.THRESH_BINARY)

    gray = cv2.GaussianBlur(gray_frame, (7,7), 0)  # фильтр Гауссиана для уменьшения шума
    _, bw_mask = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY_INV)
    combined_mask = cv2.bitwise_and(fgmask, bw_mask)

    # Морфологические операции для улучшения маски
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9)) # Отдельное ядро для предобработки
    gray = cv2.erode(gray, kernel, iterations=2)  # Сжатие для удаления мелких  шумов / затемнение
    gray = cv2.dilate(gray, kernel, iterations=2)  # Расширение для соединения частей объекта / осветление

    combined_mask = cv2.bitwise_and(fgmask, bw_mask)  # Комбинированная маска
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel, iterations=3) # Убирает шум
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel, iterations=3) # Закрывает дыры

    # Поиск и фильтрация контуров
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    valid_contours = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < min_area:
            continue
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / h
        if aspect_ratio > max_aspect_ratio or aspect_ratio < 1/max_aspect_ratio:
            continue
        valid_contours.append(cnt)

    return combined_mask, valid_contours

# Отрисовка траектории на кадре

In [ ]:
def draw_trajectory(frame, trajectory, color=(255, 0, 255), thickness=2):
    if len(trajectory) > 1:
        pts = np.array(trajectory, np.int32).reshape((-1, 1, 2))
        cv2.polylines(frame, [pts], False, color, thickness) # Рисуем на оригинальном кадре

# Основная функция трекинга мыши


In [ ]:
def track_mouse(video_path, output_gif, mask_gif_path,
               var_threshold=10, # Порог чувствительности вычитания фона (меньше = чувствительнее)
               detection_interval=12,  # Кадров между полными циклами обнаружения
               min_area=3000, # Минимальная площадь объекта для обнаружения (фильтрация шума)
               max_aspect_ratio=4.0,  # Максимальное соотношение сторон (фильтрация продолговатых объектов)
               scale_factor=0.25 # Коэффициент уменьшения (0.5 = 50% от исходного размера изображения)
              ):

    cap, original_width, original_height = initialize_video(video_path) # Открываем видеофайл

    # Настройка вычитателя фона
    background = calculate_static_background(
      cap,
      initial_frames=30,
      learning_rate=0.1,
      median_blur=7,          # Усиленное размытие
      morph_iterations=3      # Больше морфологии
      )

    # Переменные состояния
    tracker_state = None # Переменная для хранения трекера
    trajectory = [] # Хранение координат траектории
    frames = [] # Список для хранения обработанных кадров
    mask_frames = [] # список для хранения масок


    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Обработка трекера
        if tracker_state is not None:
            tracker_state, bbox, success = csrt_tracking(frame, bbox, tracker_state)
            if success:
                x, y, w, h = map(int, bbox)         # Получаем координаты
                cx = x + w//2                       # Центр объекта по X
                cy = y + h//2                       # Центр объекта по Y

                SMOOTHING_FACTOR = 0.3  # Коэффициент сглаживания (0.0 - максимальное сглаживание, 1.0 - без сглаживания)
                MAX_SMOOTH_POINTS = 3   # Количество точек для усреднения

                # Плавное обновление координат
                if len(trajectory) >= MAX_SMOOTH_POINTS:
                  # Берем среднее из последних MAX_SMOOTH_POINTS точек
                  smooth_cx = int(np.mean([p[0] for p in trajectory[-MAX_SMOOTH_POINTS:]]) * (1 - SMOOTHING_FACTOR) + cx * SMOOTHING_FACTOR)
                  smooth_cy = int(np.mean([p[1] for p in trajectory[-MAX_SMOOTH_POINTS:]]) * (1 - SMOOTHING_FACTOR) + cy * SMOOTHING_FACTOR)
                  cx, cy = smooth_cx, smooth_cy

                trajectory.append((cx, cy))

                # Отрисовка трекера
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2) # Прямоугольник вокруг объекта
                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1) # Центр объекта
                cv2.putText(frame, "Tracking", (x, y-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                tracker_state = None # Сброс трекера при потере

        # Периодическое обнаружение объекта
        if not tracker_state or len(trajectory) % detection_interval == 0:
            mask, valid_contours = process_frame(frame, background, var_threshold,
                                   min_area, max_aspect_ratio)

            if valid_contours:
                main_contour = max(valid_contours, key=cv2.contourArea)
                x, y, w, h = cv2.boundingRect(main_contour)
                tracker_state, bbox, _ = csrt_tracking(frame, (x,y,w,h))  # Инициализация

        # Сохранение маски и кадра
        mask_frames.append(cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
                          if 'mask' in locals() else np.zeros_like(frame))

        draw_trajectory(frame, trajectory)
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Сохранение результатов
    size = (int(original_width*scale_factor), int(original_height*scale_factor))
    with imageio.get_writer(output_gif, duration=0.03, loop=0) as writer:
        for frame in frames:
            writer.append_data(cv2.resize(frame, size)) # Создание GIF

    with imageio.get_writer(mask_gif_path, duration=0.03, loop=0) as writer:
        for mask in mask_frames:
            writer.append_data(cv2.resize(mask, size)) # Сохраняем маски в отдельный GIF

    cap.release() # Освобождение ресурсов
    print(f"Готово! Сохранено: {output_gif} и {mask_gif_path}")

# Вызов функции

In [ ]:
track_mouse(
    video_path='/content/drive/MyDrive/Colab Notebooks/LB3_Mouse10.mp4', # Путь к исходному видео
    output_gif='/content/drive/MyDrive/Colab Notebooks/LB3_Mouse_tracking10.gif', # Путь для сохранения GIF
    mask_gif_path='/content/drive/MyDrive/Colab Notebooks/LB3_Mouse_mask10.gif' # Путь для сохранения GIF бинаризированной маски
)

Готово! Сохранено: /content/drive/MyDrive/Colab Notebooks/LB3_Mouse_tracking10.gif и /content/drive/MyDrive/Colab Notebooks/LB3_Mouse_mask10.gif
